In [2]:
import sys
sys.path.append("./estimator")
from estimator.rok_estimator.rok_estimator import *

class Relation_Poly(Relation):
    def execute(self, op, **kwargs):
        match op:
            case "norm-poly":
                return self.pi_norm_poly()
            case "norm-poly-z":
                return self.pi_norm_poly_z(**kwargs)
            case _:
                return super().execute(op, **kwargs)
            
    def pi_norm_poly(self):
        comm = (3 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.size_Rq()  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
        # comm = (2 * self.rep) * self.ring.size_Rq() + (1 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.log_q  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
        snd_err = 3 * ceil(log(self.wdim, 2))/(2**(self.ring.log_q * self.ring.residue_deg)) # TODO: Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the soundness error.
        rel_params = {
            # "ring": self.ring,
            # "trivial": self.trivial,
            "op_name": "norm-poly",
            "n_compress": self.n_compress + 2,
            # "n_commit": self.n_commit,
            "n_rel": self.n_rel + 2,
            # "wdim": self.wdim,
            # "rep": self.rep,
            # "log_beta_wit_2": bound_log_canon_2_from_log_coeff_inf(self.ring,self.log_beta_wit_inf, dim=self.wdim * (self.rep + ell)), # Measured in Frobenius norm
            # "log_beta_wit_2": self.log_beta_wit_2, # Measured in max ell_2-norm over all columns
            # "log_beta_wit_inf": self.log_beta_wit_inf
            "comm" : comm,      
            "acc_comm" : self.acc_comm + comm,       
            "snd_err" : snd_err,
            "acc_snd_err" : self.acc_snd_err + snd_err,           
            "log_beta_ext_2_func" : lambda x : self.log_beta_wit_2, # perfect extraction
            "log_beta_ext_inf_func" : lambda x : oo # extraction of ell_2-norm is perfect, then bound ell_inf-norm by norm conversion
        }      
        return replace(self, **rel_params)


class Simulation_Poly(Simulation):
    def check_norm(self):
        # Backward direction, a.k.a. "extraction direction"        
        for i in range(len(self.trace)-1):
            rel_tgt = self.trace[-i-1]
            rel_src = self.trace[-i-2]
            
            if self.trace[-i-1].op_name == "norm":
                if rel_src.ring.log_q - 1 <= rel_src.log_beta_ext_inf * 2:
                    self.error_log += [f"TRUE Extraction failure for pi_norm: The norm of the square of the extracted witness is 2^{ceil(rel_src.log_beta_ext_inf * 2 + log(rel_src.ring.ring_exp_inf,2) + log(rel_src.wdim,2))} overflowing modulo q." + str(i)]

            if self.trace[-i-1].op_name == "norm-poly":
                if rel_src.ring.log_q - 1 <= rel_src.log_beta_ext_inf * 2:
                    self.error_log += [f" Extraction failure for pi_norm: The norm of the square of the extracted witness is 2^{ceil(rel_src.log_beta_ext_inf * 2 + log(rel_src.ring.ring_exp_inf,2) + log(rel_src.wdim,2))} overflowing modulo q." + str(i)]


In [4]:
def print_log():
    print("""
            pub const PHI: usize = {phi};
            pub static MODULE_SIZE: usize = {n_sis};
            pub static WIT_DIM: usize = {wit_dim};
            pub static REP: usize = {rep};
            pub static INIT_REP: usize = {rep};
            pub static NOF_ROUNDS: i32 = {rounds};
            pub static DECOMP_PARTS: [usize; {rounds}] = {radices};
            """.format(phi=ring_params["f"]-1, rounds=len(radices), radices=radices, rep=rel_params["rep"], wit_dim=rel_params["wdim"], n_sis=sim.ring.n_sis))

def ops_from_radices(radices):
    big_loop = lambda ell: [("bdecomp", {"ell": ell }), ("norm-poly", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {})]
    
    ops_w = list(map(lambda r: big_loop(r),radices))
    
    ops = [
        x
        for xs in ops_w
        for x in xs
    ] + [("finish", {})]

    return ops

def d_ops_from_radices(radices):
    big_loop = lambda ell: [("norm-poly", {}), ("bdecomp", {"ell": ell }), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {})]
    
    ops_w = list(map(lambda r: big_loop(r),radices))
    
    ops = [
        x
        for xs in ops_w
        for x in xs
    ] + [("finish", {})]

    return ops


def old_ops_from_radices(radices):
    big_loop = lambda ell: [("bdecomp", {"ell": ell }), ("norm", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {})]
    
    ops_w = list(map(lambda r: big_loop(r),radices))
    
    ops = [
        x
        for xs in ops_w
        for x in xs
    ] + [("finish", {})]

    return ops
        

In [45]:
# OLD A 
ring_params = {
    "f": 17, # this is perfect so we multiply ofer f=64
    "log_beta_sis_2": 40.5,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**23,
    "rep": 22,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}

radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = old_ops_from_radices(radices)

sim = Simulation(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.show()
print_log()


Ring parameters:
    conductor f: 17, degree phi: 16, modulus q: 2^62, beta_sis_2: 2^40.5000000000000
    SIS module rank n_sis: 65, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 124.0 B, |R_q^(n_sis)| = 7.871 KB
 
Subtractive set parameters:
    cardinality: 17
    forward ell_2 expansion factor gamma_2: 2^5
    inverse ell_2 expansion factor theta_2: 2^3
    forward ell_inf expansion factor gamma_inf: 2^4
    inverse ell_inf expansion factor theta_inf: 2^4
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  8388608 |  22 |        ( 16 | 18 )        |         (  0 | 18 )         | 352.0 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |  8388608 |  22 |        ( 16 | 18 )        |       

In [46]:
# NEW A RERUN
ring_params = {
    "f": 17,
    "log_beta_sis_2": 38.5,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**23,
    "rep": 20,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


ops = ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()
print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 17, degree phi: 16, modulus q: 2^62, beta_sis_2: 2^38.5000000000000
    SIS module rank n_sis: 59, target SIS security: 128, resulting SIS security: 129
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 124.0 B, |R_q^(n_sis)| = 7.145 KB
 
Subtractive set parameters:
    cardinality: 17
    forward ell_2 expansion factor gamma_2: 2^5
    inverse ell_2 expansion factor theta_2: 2^3
    forward ell_inf expansion factor gamma_inf: 2^4
    inverse ell_inf expansion factor theta_inf: 2^4
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  8388608 |  20 |        ( 16 | 16 )        |         (  0 | 16 )         | 320.0 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |  8388608 |  20 |        ( 16 | 16 )        |       

In [47]:
# OLD B 
ring_params = {
    "f": 31,
    "log_beta_sis_2": 43,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**22,
    "rep": 18,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}

radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,2]

ops = old_ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 31, degree phi: 30, modulus q: 2^62, beta_sis_2: 2^43
    SIS module rank n_sis: 39, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 232.5 B, |R_q^(n_sis)| = 8.855 KB
 
Subtractive set parameters:
    cardinality: 31
    forward ell_2 expansion factor gamma_2: 2^5
    inverse ell_2 expansion factor theta_2: 2^4
    forward ell_inf expansion factor gamma_inf: 2^5
    inverse ell_inf expansion factor theta_inf: 2^5
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  4194304 |  18 |        ( 16 | 19 )        |         (  0 | 19 )         | 270.0 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |  4194304 |  18 |        ( 16 | 19 )        |         (  0 | 19 ) 

In [48]:
# NEW B OK
ring_params = {
    "f": 31,
    "log_beta_sis_2": 39.9,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**22,
    "rep": 18,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}

radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,2]

ops = ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 31, degree phi: 30, modulus q: 2^62, beta_sis_2: 2^39.9000000000000
    SIS module rank n_sis: 34, target SIS security: 128, resulting SIS security: 130
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 232.5 B, |R_q^(n_sis)| = 7.720 KB
 
Subtractive set parameters:
    cardinality: 31
    forward ell_2 expansion factor gamma_2: 2^5
    inverse ell_2 expansion factor theta_2: 2^4
    forward ell_inf expansion factor gamma_inf: 2^5
    inverse ell_inf expansion factor theta_inf: 2^5
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  4194304 |  18 |        ( 16 | 16 )        |         (  0 | 16 )         | 270.0 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |  4194304 |  18 |        ( 16 | 16 )        |       

In [49]:
# OLD C with relaxed condition OK
ring_params = {
    "f": 113,
    "log_beta_sis_2": 48.5,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**20,
    "rep": 14,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}

radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = old_ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 113, degree phi: 112, modulus q: 2^62, beta_sis_2: 2^48.5000000000000
    SIS module rank n_sis: 14, target SIS security: 128, resulting SIS security: 136
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 868.0 B, |R_q^(n_sis)| = 11.87 KB
 
Subtractive set parameters:
    cardinality: 113
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^6
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  1048576 |  14 |        ( 17 | 19 )        |         (  0 | 19 )         | 196.0 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |  1048576 |  14 |        ( 17 | 19 )        |    

In [50]:
# NEW C 
ring_params = {
    "f": 113,
    "log_beta_sis_2": 45.6,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**20,
    "rep": 14,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}

radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 113, degree phi: 112, modulus q: 2^62, beta_sis_2: 2^45.6000000000000
    SIS module rank n_sis: 12, target SIS security: 128, resulting SIS security: 131
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 868.0 B, |R_q^(n_sis)| = 10.17 KB
 
Subtractive set parameters:
    cardinality: 113
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^6
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  1048576 |  14 |        ( 17 | 17 )        |         (  0 | 17 )         | 196.0 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |  1048576 |  14 |        ( 17 | 17 )        |    

In [51]:
# OLD D with relaxed conditions RERUN
ring_params = {
    "f": 251,
    "log_beta_sis_2": 52.1,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**19,
    "rep": 12,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}

radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = old_ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 251, degree phi: 250, modulus q: 2^62, beta_sis_2: 2^52.1000000000000
    SIS module rank n_sis: 7, target SIS security: 128, resulting SIS security: 131
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 1.892 KB, |R_q^(n_sis)| = 13.24 KB
 
Subtractive set parameters:
    cardinality: 251
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^7
    forward ell_inf expansion factor gamma_inf: 2^8
    inverse ell_inf expansion factor theta_inf: 2^8
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   524288 |  12 |        ( 18 | 20 )        |         (  0 | 20 )         | 187.5 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   524288 |  12 |        ( 18 | 20 )        |    

In [52]:
# NEW D OK
ring_params = {
    "f": 251,
    "log_beta_sis_2": 49.2,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**19,
    "rep": 12,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}

radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 251, degree phi: 250, modulus q: 2^62, beta_sis_2: 2^49.2000000000000
    SIS module rank n_sis: 7, target SIS security: 128, resulting SIS security: 149
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 1.892 KB, |R_q^(n_sis)| = 13.24 KB
 
Subtractive set parameters:
    cardinality: 251
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^7
    forward ell_inf expansion factor gamma_inf: 2^8
    inverse ell_inf expansion factor theta_inf: 2^8
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   524288 |  12 |        ( 18 | 18 )        |         (  0 | 18 )         | 187.5 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   524288 |  12 |        ( 18 | 18 )        |    

In [53]:
# OLD E OK
ring_params = {
    "f": 509,
    "log_beta_sis_2": 54.5,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**19,
    "rep": 10,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2]

ops = old_ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 509, degree phi: 508, modulus q: 2^62, beta_sis_2: 2^54.5000000000000
    SIS module rank n_sis: 4, target SIS security: 128, resulting SIS security: 140
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 3.845 KB, |R_q^(n_sis)| = 15.38 KB
 
Subtractive set parameters:
    cardinality: 509
    forward ell_2 expansion factor gamma_2: 2^9
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^9
    inverse ell_inf expansion factor theta_inf: 2^9
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   524288 |  10 |        ( 19 | 21 )        |         (  0 | 21 )         | 317.5 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   524288 |  10 |        ( 19 | 21 )        |    

In [54]:
# NEW E OK
ring_params = {
    "f": 509,
    "log_beta_sis_2": 52.8,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**19,
    "rep": 10,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}

radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 509, degree phi: 508, modulus q: 2^62, beta_sis_2: 2^52.8000000000000
    SIS module rank n_sis: 4, target SIS security: 128, resulting SIS security: 151
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 3.845 KB, |R_q^(n_sis)| = 15.38 KB
 
Subtractive set parameters:
    cardinality: 509
    forward ell_2 expansion factor gamma_2: 2^9
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^9
    inverse ell_inf expansion factor theta_inf: 2^9
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   524288 |  10 |        ( 19 | 19 )        |         (  0 | 19 )         | 317.5 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   524288 |  10 |        ( 19 | 19 )        |    

In [55]:
# OLD F OK
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 57.2,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**18,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2]

ops = old_ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^57.2000000000000
    SIS module rank n_sis: 3, target SIS security: 128, resulting SIS security: 200
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 23.16 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   262144 |   9 |        ( 19 | 21 )        |         (  0 | 21 )         | 286.9 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   262144 |   9 |        ( 19 | 21 )       

In [56]:
# NEW F OK
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 55.9,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**18,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^55.9000000000000
    SIS module rank n_sis: 2, target SIS security: 128, resulting SIS security: 133
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 15.44 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   262144 |   9 |        ( 19 | 19 )        |         (  0 | 19 )         | 286.9 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   262144 |   9 |        ( 19 | 19 )       

In [50]:
# OLD F_2
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 56.2,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**17,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2]

ops = old_ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^56.2000000000000
    SIS module rank n_sis: 2, target SIS security: 128, resulting SIS security: 132
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 15.44 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   131072 |   9 |        ( 19 | 21 )        |         (  0 | 21 )         | 143.4 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   131072 |   9 |        ( 19 | 21 )       

In [17]:
# NEW F_2 
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 56.9,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**17,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^56.9000000000000
    SIS module rank n_sis: 2, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 15.44 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   131072 |   9 |        ( 19 | 19 )        |         (  0 | 19 )         | 143.4 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   131072 |   9 |        ( 19 | 19 )       

In [39]:
# OLD F_3
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 56.2,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**16,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 3, 3, 3, 3, 2, 2, 2, 2, 2]

ops = old_ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^56.2000000000000
    SIS module rank n_sis: 2, target SIS security: 128, resulting SIS security: 132
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 15.44 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |    65536 |   9 |        ( 18 | 20 )        |         (  0 | 20 )         | 71.72 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |    65536 |   9 |        ( 18 | 20 )       

In [41]:
# NEW F_3
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 56.9,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**16,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^56.9000000000000
    SIS module rank n_sis: 2, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 15.44 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |    65536 |   9 |        ( 18 | 18 )        |         (  0 | 18 )         | 71.72 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |    65536 |   9 |        ( 18 | 18 )       

In [44]:
# OLD F_4
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 56.2,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**15,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 3, 3, 3, 2, 2, 2, 2, 2]

ops = old_ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^56.2000000000000
    SIS module rank n_sis: 2, target SIS security: 128, resulting SIS security: 132
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 15.44 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |    32768 |   9 |        ( 18 | 20 )        |         (  0 | 20 )         | 35.86 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |    32768 |   9 |        ( 18 | 20 )       

In [49]:
# NEW F_4
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 56.9,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**15,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 2, 2, 2, 2, 2, 2, 2, 2]

ops = ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^56.9000000000000
    SIS module rank n_sis: 2, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 15.44 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |    32768 |   9 |        ( 18 | 18 )        |         (  0 | 18 )         | 35.86 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |    32768 |   9 |        ( 18 | 18 )       

In [52]:
# OLD F_0 
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 57.2,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**19,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2,2]

ops = old_ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^57.2000000000000
    SIS module rank n_sis: 3, target SIS security: 128, resulting SIS security: 200
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 23.16 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   524288 |   9 |        ( 20 | 22 )        |         (  0 | 22 )         | 573.8 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   524288 |   9 |        ( 20 | 22 )       

In [57]:
# NEW F_0 OK
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 55.9,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**19,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^55.9000000000000
    SIS module rank n_sis: 2, target SIS security: 128, resulting SIS security: 133
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 15.44 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |   524288 |   9 |        ( 20 | 20 )        |         (  0 | 20 )         | 573.8 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |   524288 |   9 |        ( 20 | 20 )       

In [64]:
# OLD F_00 
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 57.2,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**20,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2,2]

ops = old_ops_from_radices(radices)


sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^57.2000000000000
    SIS module rank n_sis: 3, target SIS security: 128, resulting SIS security: 200
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 23.16 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  1048576 |   9 |        ( 20 | 22 )        |         (  0 | 22 )         | 1148. MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |  1048576 |   9 |        ( 20 | 22 )       

In [72]:
# NEW F_00 OK
ring_params = {
    "f": 1021,
    "log_beta_sis_2": 56,
    "log_q": 62,
}

rel_params = {
    "wdim": 2**20,
    "rep": 9,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "d": 2,
    "ell": 3
}


radices = [1, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2]

ops = ops_from_radices(radices)

sim = Simulation_Poly(ring_params, rel_params, Relation_Poly)
print(sim.ring)
sim.execute(ops)
sim.extract()
sim.check_norm()
sim.show()

print(float(sim.max_log_beta_ext_2))
print_log()

Ring parameters:
    conductor f: 1021, degree phi: 1020, modulus q: 2^62, beta_sis_2: 2^56
    SIS module rank n_sis: 2, target SIS security: 128, resulting SIS security: 133
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 7.720 KB, |R_q^(n_sis)| = 15.44 KB
 
Subtractive set parameters:
    cardinality: 1021
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^10
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  1048576 |   9 |        ( 20 | 20 )        |         (  0 | 20 )         | 1148. MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   |  1048576 |   9 |        ( 20 | 20 )        |         (  